In [ ]:
! pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 49.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=414efd6e60bdc06e6c5ab7fda8164b813e7ac4ed4feb8fb6cc354b3f8e3f8b33
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir("/content/drive/MyDrive")

In [ ]:
! unrar e 1k.rar -C /content/


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from 1k.rar

Extracting  /content/1k.csv                                               99%  OK 
All OK


In [ ]:
os.chdir("/content/")

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.driver.memory", "15g") \
    .appName("Train_ml") \
    .getOrCreate()

train = spark.read.options(header='true').csv("1k.csv")

missing = train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns])
null_counts = missing.collect()[0].asDict()
to_drop = [k for k, v in null_counts.items() if v > (1000*0.001)]
print(len(to_drop))
print(to_drop)
train = train.drop(*to_drop)
#train = train.sample(0.1)
print(len(train.columns))
print(train.count())

In [ ]:
raw_features = train.columns
raw_features.remove('isFraud')

categorical = ['ProductCD',
               'card1','card2','card3','card4','card5','card6', 
               'addr1', 'addr2',
               'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
               'P_emaildomain', 'R_emaildomain',
               'DeviceType','DeviceInfo',
               'id_12',  'id_13', 'id_14',  'id_15',
               'id_16',  'id_17',  'id_18',  'id_19',  'id_20',
               'id_21',  'id_22',  'id_23',  'id_24',  'id_25',
               'id_26',  'id_27',  'id_28',  'id_29',  'id_30',
               'id_31',  'id_32',  'id_33',  'id_34',  'id_35',  'id_36',  'id_37',  'id_38']
categorical = list(set(categorical) - set(to_drop))
print(categorical)


['card6', 'card3', 'card4', 'ProductCD', 'card1']


In [ ]:
numerical = list(set(raw_features) - set(categorical))



for col_name in numerical:
    train = train.withColumn(col_name, col(col_name).cast('float'))
    
train = train.withColumn('isFraud', col('isFraud').cast('float'))

train.printSchema()

root
 |-- TransactionID: float (nullable = true)
 |-- isFraud: float (nullable = true)
 |-- TransactionDT: float (nullable = true)
 |-- TransactionAmt: float (nullable = true)
 |-- ProductCD: string (nullable = true)
 |-- card1: string (nullable = true)
 |-- card3: string (nullable = true)
 |-- card4: string (nullable = true)
 |-- card6: string (nullable = true)
 |-- C1: float (nullable = true)
 |-- C2: float (nullable = true)
 |-- C3: float (nullable = true)
 |-- C4: float (nullable = true)
 |-- C5: float (nullable = true)
 |-- C6: float (nullable = true)
 |-- C7: float (nullable = true)
 |-- C8: float (nullable = true)
 |-- C9: float (nullable = true)
 |-- C10: float (nullable = true)
 |-- C11: float (nullable = true)
 |-- C12: float (nullable = true)
 |-- C13: float (nullable = true)
 |-- C14: float (nullable = true)
 |-- D1: float (nullable = true)
 |-- V95: float (nullable = true)
 |-- V96: float (nullable = true)
 |-- V97: float (nullable = true)
 |-- V98: float (nullable = true)

In [ ]:
categorical_output = ['card4_index', 'card6_index', 'ProductCD_index', 'card1_index', 'card3_index']
'''
categorical_output = ['ProductCD_index',
               'card1_index','card2_index','card3_index','card4_index','card5_index','card6_index', 
               'addr1_index', 'addr2_index',
               'M1_index', 'M2_index', 'M3_index', 'M4_index', 'M5_index', 'M6_index', 'M7_index', 'M8_index', 'M9_index',
               'P_emaildomain_index', 'R_emaildomain_index',
               'DeviceType_index','DeviceInfo_index',
               'id12_index',  'id13_index', 'id14_index',  'id15_index',
               'id16_index',  'id17_index',  'id18_index',  'id19_index',  'id20_index',
               'id21_index',  'id22_index',  'id23_index',  'id24_index',  'id25_index',
               'id26_index',  'id27_index',  'id28_index',  'id29_index',  'id30_index',
               'id31_index',  'id32_index',  'id33_index',  'id34_index',  'id35_index',  'id36_index',  'id37_index',  'id38_index']
'''

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=categorical, outputCols=categorical_output, handleInvalid="skip")
#categorical_indexed = indexer.fit(train).transform(train)

#categorical_indexed.printSchema()
#categorical_indexed.count()

In [ ]:
from pyspark.ml.feature import VectorAssembler

all_features = categorical_output + numerical
assembler = VectorAssembler(
    inputCols=all_features,
    outputCol="features",handleInvalid="skip")
#train_vector = assembler.transform(categorical_indexed)

In [ ]:
from pyspark.ml.feature import PCA
pca = PCA(k=30, inputCol='features', outputCol='pcaFeature')

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="isFraud", featuresCol="pcaFeature")

#model = lr.fit(trainingData)

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer,assembler,pca,lr])

In [ ]:
(trainingData, testData) = train.randomSplit([0.7, 0.3])
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="isFraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Logistic Regression Accuracy = %g" % (accuracy))

Test Accracy = 0.925926


In [65]:
# cross-fold
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

lr = LogisticRegression(labelCol="isFraud")
pipeline = Pipeline(stages=[indexer,assembler,pca,lr])

grid = ParamGridBuilder().addGrid(lr.maxIter, [0, 2]).build()
evaluator = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="accuracy")

cv = CrossValidator(estimator=pipeline, 
           estimatorParamMaps=grid,
           evaluator=evaluator,
           numFolds=5,
           parallelism=2)
cvModel = cv.fit(trainingData) # run the crossValidation and automatically select the best combination of paramerters

prediction = cvModel.transform(testData)
evaluator.evaluate(prediction)

In [67]:
cvModel.avgMetrics #validation accuracy

[0.9681563079568576, 0.9650313079568578]

In [68]:
evaluator.evaluate(prediction) #test accuracy

0.9444444444444444

In [74]:
# Print the best combination of arguments
parameters = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        cvModel.getEstimatorParamMaps(), 
        cvModel.avgMetrics
    )
]
 
print(sorted(parameters, key=lambda el: el[1], reverse=True)[0])

([{'maxIter': 0}], 0.9681563079568576)


In [121]:
# decision tree
from pyspark.ml.classification import DecisionTreeClassifier

dt =  DecisionTreeClassifier(maxDepth=2,labelCol="isFraud", featuresCol="pcaFeature") 
dt_pipeline = Pipeline(stages=[indexer,assembler,pca,dt]) 
dt_model = dt_pipeline.fit(trainingData)
dt_predictions = dt_model.transform(testData)

dt_accuracy = evaluator.evaluate(dt_predictions)
print("Test Decision Tree Accuracy = %g" % (dt_accuracy))

Test Decision Tree Accuracy = 0.944444


In [23]:
# random forest
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="isFraud",seed=42,featuresCol="pcaFeature")
rf_pipeline = Pipeline(stages=[indexer,assembler,pca,rf]) 
rf_model = rf_pipeline.fit(trainingData)
rf_predictions = rf_model.transform(testData)

rf_accuracy = evaluator.evaluate(rf_predictions)
print("Test Random Forest Accuracy = %g" % (rf_accuracy))

Test Accracy = 0.944444


In [114]:
# According to materials, Check layer size for MLP, but this is not used
test_pipeline = Pipeline(stages=[indexer,assembler,pca])
test_model = test_pipeline.fit(trainingData)
test = test_model.transform(trainingData)
test.schema["features"].metadata["ml_attr"]["num_attrs"]

109

In [119]:
# nn - MLP
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [30,5,3,2] # 30 is set according to error message given by code, 2 stands for isFraud or not
trainer = MultilayerPerceptronClassifier(featuresCol="pcaFeature", labelCol="isFraud", maxIter=100, layers=layers, blockSize=128, seed=1234)
mlp_pipeline = Pipeline(stages=[indexer,assembler,pca,trainer])
mlp_model = mlp_pipeline.fit(trainingData)
mlp_predictions = mlp_model.transform(testData)

predictionAndLabels = mlp_predictions.select("prediction", "isFraud")
mlp_accuracy = evaluator.evaluate(mlp_predictions)
print("Test Neural Network Accuracy = %g" % (mlp_accuracy))

Test Neural Network Accuracy = 0.944444


In [122]:
# linear svm
from pyspark.ml.classification import LinearSVC

SVC = LinearSVC(featuresCol="pcaFeature", labelCol="isFraud")
svc_pipeline = Pipeline(stages=[indexer,assembler,pca,SVC])
svc_model = svc_pipeline.fit(trainingData)
svc_predictions = svc_model.transform(testData)

svc_accuracy = evaluator.evaluate(svc_predictions)
print("Test Linear SVM Accuracy = %g" % (svc_accuracy))

Test Linear SVM Accuracy = 0.944444


In [33]:
# naive bayes
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
nb_pipeline = Pipeline(stages=[indexer,assembler,pca,rf]) 
nb_model = nb_pipeline.fit(trainingData)
nb_predictions = nb_model.transform(testData)

nb_accuracy = evaluator.evaluate(nb_predictions)
print("Test Naive Beyes Accuracy = %g" % (nb_accuracy))

Test Accracy = 0.944444
